In [1]:
import pandas as pd

df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")

print("Loaded")
print("Dataset shape:", df.shape)

df.head()


Loaded
Dataset shape: (1470, 35)


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [5]:
import sqlite3
conn = sqlite3.connect("hr.db")
df.to_sql("employees", conn, if_exists="replace", index=False)
print("Loaded:", df.shape)

Loaded: (1470, 35)


In [9]:
total_employees = df.shape[0]
total_employees


1470

In [12]:
pd.read_sql_query("""
SELECT COUNT(*) AS total_employees
FROM employees;
""", conn)

,total_employees
0,1470


In [13]:
dept_counts = df["Department"].value_counts()
dept_counts


Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [14]:
pd.read_sql_query("""
SELECT Department, COUNT(*) AS employee_count
FROM employees
GROUP BY Department
ORDER BY employee_count DESC;
""", conn)

,Department,employee_count
0,Research & Development,961
1,Sales,446
2,Human Resources,63


In [16]:
avg_income_role = df.groupby("JobRole")["MonthlyIncome"].mean().sort_values(ascending=False)
avg_income_role.round(2)


JobRole
Manager                      17181.68
Research Director            16033.55
Healthcare Representative     7528.76
Manufacturing Director        7295.14
Sales Executive               6924.28
Human Resources               4235.75
Research Scientist            3239.97
Laboratory Technician         3237.17
Sales Representative          2626.00
Name: MonthlyIncome, dtype: float64

In [17]:
pd.read_sql_query("""
SELECT JobRole, ROUND(AVG(MonthlyIncome), 2) AS avg_monthly_income
FROM employees
GROUP BY JobRole
ORDER BY avg_monthly_income DESC;
""", conn)

,JobRole,avg_monthly_income
0,Manager,17181.68
1,Research Director,16033.55
2,Healthcare Representative,7528.76
3,Manufacturing Director,7295.14
4,Sales Executive,6924.28
5,Human Resources,4235.75
6,Research Scientist,3239.97
7,Laboratory Technician,3237.17
8,Sales Representative,2626.00


In [18]:
top5_perf = df.sort_values(
    by=["PerformanceRating", "MonthlyIncome"],
    ascending=[False, False]
).head(5)

top5_perf[["EmployeeNumber", "PerformanceRating", "MonthlyIncome"]]


,EmployeeNumber,PerformanceRating,MonthlyIncome
746,1035,4,19973
918,1282,4,19847
1009,1423,4,19701
1129,1595,4,19613
1116,1578,4,19586


In [19]:
pd.read_sql_query("""
SELECT EmployeeNumber, PerformanceRating, MonthlyIncome
FROM employees
ORDER BY PerformanceRating DESC, MonthlyIncome DESC
LIMIT 5;
""", conn)

,EmployeeNumber,PerformanceRating,MonthlyIncome
0,1035,4,19973
1,1282,4,19847
2,1423,4,19701
3,1595,4,19613
4,1578,4,19586


In [25]:
dept_perf = df.groupby("Department")["PerformanceRating"].mean().sort_values(ascending=False)
dept_perf.head(1).round(2)


Department
Research & Development    3.16
Name: PerformanceRating, dtype: float64

In [23]:
pd.read_sql_query("""
SELECT Department, ROUND(AVG(PerformanceRating), 2) AS avg_perf_rating
FROM employees
GROUP BY Department
ORDER BY avg_perf_rating DESC
LIMIT 1;
""", conn)

,Department,avg_perf_rating
0,Research & Development,3.16
